In [1]:
%matplotlib inline
# Enabling reloading extension to use external python files
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
from datetime import date, datetime,timedelta
import math
import columns
import datasets
import seasons
import directory_utils

In [3]:
# Setup matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')

In [4]:
data_frame = pd.read_csv(datasets.FIRST_CLEAN_DATA_SET_111_PATH, encoding=datasets.DEFAULT_CSV_ENCODING)

/home/felipepedroso/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data_frame.shape

(1046641, 19)

In [6]:
data_frame.columns

Index(['Cod', 'Ano', 'Dia Juliano', 'Hora - minuto',
       'Veloc. Instantânea do Vento a 5 metros (m/s)',
       'Veloc. Instantânea do Vento a 2 metros (m/s)',
       'Direção do Vento no instante da aquisição (°)',
       'Radiação Incidente total de 10 leituras (KW/m2)',
       'Radiação Refletida Total de 10 leituras (KW/m2)',
       'Fluxo Positivo de calor no solo total de 10 leituras (W/m2)',
       'Fluxo negativo de calor no solo . total de 10 leituras (W/m2)',
       'Umidade relativa Média (%)', 'Temperatura do Ar Média (°C)',
       'Temperatura do solo  1 Média (°C) à 3cm no solo',
       'Temperatura do solo 2Média (°C) à 6cm no solo ',
       'Temperatura do solo 3Média (°C) Colocado sensor de UV em 31/07/09',
       'Pressão (hPa)', 'Chuva total do período (mm)', 'Data'],
      dtype='object')

### Extracting daily information

In [7]:
daily_analysis_columns = [columns.TEMPERATURE,columns.HUMIDITY,columns.PRESSURE]

In [8]:
daily_average_from_columns = data_frame.groupby(columns.DATE)[daily_analysis_columns].mean()
daily_average_from_columns.head()

,Temperatura do Ar Média (°C),Umidade relativa Média (%),Pressão (hPa)
Data,,,
1997-01-01,22.875903,80.237222,942.916667
1997-01-02,21.545625,79.303194,943.861111
1997-01-03,20.179444,82.070000,945.409722
1997-01-04,22.125833,83.748333,943.194444
1997-01-05,23.178958,84.473542,942.729167


In [9]:
daily_max_from_columns = data_frame.groupby(columns.DATE)[daily_analysis_columns].max()
new_column_names = [columns.MAX_TEMP,columns.MAX_HUMIDITY,columns.MAX_PRESSURE]
daily_max_from_columns.columns = new_column_names
daily_max_from_columns.head()

,Temperatura do Ar Máxima (°C),Umidade relativa Máxima (%),Pressão Máxima (hPa)
Data,,,
1997-01-01,29.39,98.1,945.0
1997-01-02,25.93,88.7,947.0
1997-01-03,25.17,95.0,947.0
1997-01-04,29.21,100.0,946.0
1997-01-05,30.46,100.0,945.0


In [10]:
daily_min_from_columns = data_frame.groupby(columns.DATE)[daily_analysis_columns].min()
new_column_names = [columns.MIN_TEMP,columns.MIN_HUMIDITY,columns.MIN_PRESSURE]
daily_min_from_columns.columns = new_column_names
daily_min_from_columns.head()

,Temperatura do Ar Mínima (°C),Umidade relativa Mínima (%),Pressão Mínima (hPa)
Data,,,
1997-01-01,18.53,58.77,940.0
1997-01-02,19.21,66.41,942.0
1997-01-03,17.88,67.79,944.0
1997-01-04,17.85,56.78,941.0
1997-01-05,18.06,57.30,940.0


In [11]:
daily_analysis_data_frame = pd.merge(daily_max_from_columns,daily_min_from_columns, on=columns.DATE)
daily_analysis_data_frame = pd.merge(daily_analysis_data_frame, daily_average_from_columns, on=columns.DATE)
daily_analysis_data_frame.head()

,Temperatura do Ar Máxima (°C),Umidade relativa Máxima (%),Pressão Máxima (hPa),Temperatura do Ar Mínima (°C),Umidade relativa Mínima (%),Pressão Mínima (hPa),Temperatura do Ar Média (°C),Umidade relativa Média (%),Pressão (hPa)
Data,,,,,,,,,
1997-01-01,29.39,98.1,945.0,18.53,58.77,940.0,22.875903,80.237222,942.916667
1997-01-02,25.93,88.7,947.0,19.21,66.41,942.0,21.545625,79.303194,943.861111
1997-01-03,25.17,95.0,947.0,17.88,67.79,944.0,20.179444,82.070000,945.409722
1997-01-04,29.21,100.0,946.0,17.85,56.78,941.0,22.125833,83.748333,943.194444
1997-01-05,30.46,100.0,945.0,18.06,57.30,940.0,23.178958,84.473542,942.729167


In [12]:
daily_analysis_data_frame[columns.VARIATION_TEMP] = daily_analysis_data_frame[columns.MAX_TEMP]-daily_analysis_data_frame[columns.MIN_TEMP]
daily_analysis_data_frame[columns.VARIATION_HUMIDITY] = daily_analysis_data_frame[columns.MAX_HUMIDITY]-daily_analysis_data_frame[columns.MIN_HUMIDITY]
daily_analysis_data_frame[columns.VARIATION_PRESSURE] = daily_analysis_data_frame[columns.MAX_PRESSURE]-daily_analysis_data_frame[columns.MIN_PRESSURE]
daily_analysis_data_frame.head()

,Temperatura do Ar Máxima (°C),Umidade relativa Máxima (%),Pressão Máxima (hPa),Temperatura do Ar Mínima (°C),Umidade relativa Mínima (%),Pressão Mínima (hPa),Temperatura do Ar Média (°C),Umidade relativa Média (%),Pressão (hPa),Variação da Temperatura do Ar (°C),Variação da Umidade relativa (%),Variação da Pressão (hPa)
Data,,,,,,,,,,,,
1997-01-01,29.39,98.1,945.0,18.53,58.77,940.0,22.875903,80.237222,942.916667,10.86,39.33,5.0
1997-01-02,25.93,88.7,947.0,19.21,66.41,942.0,21.545625,79.303194,943.861111,6.72,22.29,5.0
1997-01-03,25.17,95.0,947.0,17.88,67.79,944.0,20.179444,82.070000,945.409722,7.29,27.21,3.0
1997-01-04,29.21,100.0,946.0,17.85,56.78,941.0,22.125833,83.748333,943.194444,11.36,43.22,5.0
1997-01-05,30.46,100.0,945.0,18.06,57.30,940.0,23.178958,84.473542,942.729167,12.40,42.70,5.0


In [13]:
daily_analysis_data_frame.index = pd.to_datetime(daily_analysis_data_frame.index, format='%Y-%m-%d', utc=True)
daily_analysis_data_frame[columns.DAY_OF_YEAR] = daily_analysis_data_frame.index.map(lambda date: date.timetuple().tm_yday)
daily_analysis_data_frame[columns.YEAR] = daily_analysis_data_frame.index.map(lambda date: date.year)
daily_analysis_data_frame[columns.MONTH] = daily_analysis_data_frame.index.map(lambda date: date.month)
daily_analysis_data_frame[columns.DAY] = daily_analysis_data_frame.index.map(lambda date: date.day)
daily_analysis_data_frame[columns.SEASON] = daily_analysis_data_frame.index.map(lambda date: seasons.get_season(date))

In [14]:
# Reordering columns
_ORDER = [
    columns.YEAR,
    columns.MONTH,
    columns.DAY,
    columns.DAY_OF_YEAR,
    columns.SEASON,
    columns.TEMPERATURE, 
    columns.MAX_TEMP, 
    columns.MIN_TEMP, 
    columns.VARIATION_TEMP,
    columns.HUMIDITY, 
    columns.MAX_HUMIDITY,
    columns.MIN_HUMIDITY,
    columns.VARIATION_HUMIDITY,
    columns.PRESSURE,
    columns.MAX_PRESSURE,
    columns.MIN_PRESSURE,
    columns.VARIATION_PRESSURE
]
daily_analysis_data_frame = daily_analysis_data_frame[_ORDER]

In [15]:
directory_utils.create_if_doesnt_exist(datasets.DAILY_ANALYSIS_DIRECTORY_PATH)
daily_analysis_data_frame.to_csv(datasets.DAILY_ANALYSIS_DATA_SET_111_PATH , index = True, encoding=datasets.DEFAULT_CSV_ENCODING)

In [16]:
daily_analysis_data_frame.head()

,Ano,Mês,Dia,Dia Juliano,Estação Metereológica do Ano,Temperatura do Ar Média (°C),Temperatura do Ar Máxima (°C),Temperatura do Ar Mínima (°C),Variação da Temperatura do Ar (°C),Umidade relativa Média (%),Umidade relativa Máxima (%),Umidade relativa Mínima (%),Variação da Umidade relativa (%),Pressão (hPa),Pressão Máxima (hPa),Pressão Mínima (hPa),Variação da Pressão (hPa)
Data,,,,,,,,,,,,,,,,,
1997-01-01 00:00:00+00:00,1997,1,1,1,Verão,22.875903,29.39,18.53,10.86,80.237222,98.1,58.77,39.33,942.916667,945.0,940.0,5.0
1997-01-02 00:00:00+00:00,1997,1,2,2,Verão,21.545625,25.93,19.21,6.72,79.303194,88.7,66.41,22.29,943.861111,947.0,942.0,5.0
1997-01-03 00:00:00+00:00,1997,1,3,3,Verão,20.179444,25.17,17.88,7.29,82.070000,95.0,67.79,27.21,945.409722,947.0,944.0,3.0
1997-01-04 00:00:00+00:00,1997,1,4,4,Verão,22.125833,29.21,17.85,11.36,83.748333,100.0,56.78,43.22,943.194444,946.0,941.0,5.0
1997-01-05 00:00:00+00:00,1997,1,5,5,Verão,23.178958,30.46,18.06,12.40,84.473542,100.0,57.30,42.70,942.729167,945.0,940.0,5.0
